### Day 15 Part 1

This puzzle reminds me of a Project Euler problem I worked on (after reading someone else's solution and heavily incorporating it) years ago. 

I will be honest - I had to review that code to remind myself how this all worked. Here is the original post I worked through: https://codereview.stackexchange.com/questions/60930/project-euler-81-minimum-path-sum-through-a-matrix

Changes I made: Not much, but I am using Numpy!

### Logic: 

Below is my reasoning on the solution:

- What we need to do is determine total risk at the very last point, without really caring about the general path for part 1.
- We can do this by accumulating values over each step, starting with the row and column
    - In row 1 we update values to be: row 1 col 1, row 1 col 1 + row 1 col 2, row 1 col 1+ r1c2 + r1c3...etc
    - In col 1 we update values to be: r1c1, r1c1 + r2c1, r1c1 + r2c1 + r3c1....etc.
    - Why do this? We need an initial cumulative border given the next steps
- At this point we know the cost of moving directly across a row (it is just the sum of the row). We also know the cost of moving directly down a column (it is just the sum of the column). Likely our solution requires more movement than this. 
- We then iterate through the remaining values of our matrix (n-1 x n-1):
    - At each position we look at where we could have moved from and add the smaller value. 
    - For example, if I am looking at the value of (3,3) I could have ended up here in two ways:
        - From above, meaning my last point was (2,3) <- row 2, col 3
        - From the left, meaning my last point was (3,2) <- row 3 col 2.
        - By adding the smallest path I ensure that I make the optimal decision for this point.
- By the end our last point (n,n) is going to represent the cumulative value of our path.

In [1]:
import numpy as np
from itertools import accumulate 

# read data 
with open('data/day15_test.txt') as fh:
    data = [line.strip('\n') for line in fh.readlines()]
    
# then break down for each list into an array
single_list = []
for row in data:
    single_list.extend([int(x) for x in row])
    
# build matrix
mat = np.asarray(single_list)
mat.shape = (len(data), len(data[0]))
mat

array([[1, 1, 6, 3, 7, 5, 1, 7, 4, 2],
       [1, 3, 8, 1, 3, 7, 3, 6, 7, 2],
       [2, 1, 3, 6, 5, 1, 1, 3, 2, 8],
       [3, 6, 9, 4, 9, 3, 1, 5, 6, 9],
       [7, 4, 6, 3, 4, 1, 7, 1, 1, 1],
       [1, 3, 1, 9, 1, 2, 8, 1, 3, 7],
       [1, 3, 5, 9, 9, 1, 2, 4, 2, 1],
       [3, 1, 2, 5, 4, 2, 1, 6, 3, 9],
       [1, 2, 9, 3, 1, 3, 8, 5, 2, 1],
       [2, 3, 1, 1, 9, 4, 4, 5, 8, 1]])

In [2]:
# accumulate is pretty awesome - it solves by incremental updates over elements
list(accumulate(mat[0,:]))

[1, 2, 8, 11, 18, 23, 24, 31, 35, 37]

In [3]:
def min_path_sum(matrix):
    
    size = len(matrix)
    
    # top row gets accumulated
    matrix[0,:] = list(accumulate(matrix[0,:]))
    
    for i in range(1,size):
        matrix[i,0] += matrix[i-1,0]
        for j in range(1,size):
            matrix[i,j] += min(matrix[i-1,j], matrix[i,j-1])
    
    return matrix[size-1,size-1] - matrix[0,0]

assert(min_path_sum(mat) == 40)

In [4]:
# read data 
with open('data/day15.txt') as fh:
    data = [line.strip('\n') for line in fh.readlines()]
    
# then break down for each list into an array
single_list = []
for row in data:
    single_list.extend([int(x) for x in row])
    
# build matrix
mat = np.asarray(single_list)
mat.shape = (len(data), len(data[0]))

print(f"Total risk: {min_path_sum(mat)}")

Total risk: 429


### Part 2: Test Case

The above solution should work, but need to handle the expansion.

In [5]:
# read data 
with open('data/day15_test.txt') as fh:
    data = [line.strip('\n') for line in fh.readlines()]
    
# then break down for each list into an array
single_list = []
for row in data:
    single_list.extend([int(x) for x in row])
    
# build matrix
mat = np.asarray(single_list)
mat.shape = (len(data), len(data[0]))

In [6]:
def matrixAdd(mat, idx, col=True):
    """Properly add last matrix and return stacked"""
    i,i2,j,j2 = idx
    add_mat = mat[i:i2,j:j2] + 1
    add_mat = np.where(add_mat > 9, 1, add_mat)
    
    if col:
        return np.column_stack((mat,add_mat))
    return np.row_stack((mat,add_mat))

In [7]:
# Build out top row:
a = 0
b = 10
for i in range(4):
    mat = matrixAdd(mat, (0,10,a,b))
    a += 10
    b += 10
test = [int(x) for x in '11637517422274862853338597396444961841755517295286']

# confirm top row transormed properly 
assert(list(mat[0,:]) == test)

In [8]:
# Build out second row - pretty easy now
a = 0
b = 10
for _ in range(4):
    mat = matrixAdd(mat, (a,b,0,50), col = False)
    a += 10
    b += 10

# confirm second row transormed properly 
test = [int(x) for x in '22748628533385973964449618417555172952866628316397']
assert(list(mat[10,:]) == test)
assert(mat.shape == (50,50))
assert(min_path_sum(mat) == 315)

### Part 2: Actual Data

For some reason not working

In [9]:
# read data 
with open('data/day15.txt') as fh:
    data = [line.strip('\n') for line in fh.readlines()]
    
# then break down for each list into an array
single_list = []
for row in data:
    single_list.extend([int(x) for x in row])
    
# build matrix
mat = np.asarray(single_list)
mat.shape = (len(data), len(data[0]))
print(f"Matrix shape: {mat.shape}")

Matrix shape: (100, 100)


In [10]:
# Build out top row:
a = 0
b = 100
for i in range(4):
    mat = matrixAdd(mat, (0,100,a,b))
    a += 100
    b += 100

# Build out rows 2-5
a = 0
b = 100
for _ in range(4):
    mat = matrixAdd(mat, (a,b,0,500), col = False)
    a += 100
    b += 100

# confirm values increment properly
test = []
for i in [0,100,200,300,400]:
    for j in [0,100,200,300,400]:
        test.append(mat[i, j])
    print(test)
    test = []

[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]
[4, 5, 6, 7, 8]
[5, 6, 7, 8, 9]


In [11]:
min_path_sum(mat) # wrong >|?

2851